### Machine Learning
#### MAP@K for ```K=7```

* Gonzalo Fuertes Sémelas
* Álvaro Pérez Trasancos
* Miguel Chacón Maldonado
* Ubaldo Peralta Sánchez
* Álvaro Nicholas García Crawford
* Miguel López Melgar

Load the predictions attached in the original handing, as well as the solutions:

In [1]:
import pandas as pd

# Load CSV with the predictions ("preds") and CSV with the solutions ("actual")
preds = pd.read_csv('predictions.csv')
actual = pd.read_csv('../soluciones.csv')

Create dictionaries with ```cod_persona``` as keys and a list of the contracted products as values, both for ```preds``` and ```actual```:

In [2]:
def reverse_onehot(row):
    prods = []
    for target in targets:
        if row[target] == 1:
            prods.append(target)
    return prods

targets = [col for col in preds.columns if 'ind_prod' in col]

# Create dictionary with the predictions
preds_byclient = preds.apply(reverse_onehot, axis=1)
preds_byclient.index = preds['cod_persona']
preds_byclient = preds_byclient.to_dict()

# Create dictionary with the solutions
actual_byclient = actual.apply(reverse_onehot, axis=1)
actual_byclient.index = actual['cod_persona']
actual_byclient = actual_byclient.to_dict()

In order to properly compare the predictions with the solutions, the values of the dictionaries are ordered by the key and their values converted to lists:

In [3]:
# Reorder dictionaries keys
preds_sorted = list(dict(sorted(preds_byclient.items())).values())
actual_sorted = list(dict(sorted(actual_byclient.items())).values())

The evaluation is going to be made through the metric MAP@K. Here this metric is defined:

In [4]:
def apk(actual, predicted, k):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0
    num_hits = 0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1
            score += num_hits / (i + 1)
    if not actual:
        if not predicted:
            return 1
        else:
            return 0
    return score / min(len(actual), k)

def mapk(actual, predicted, k):
    from numpy import mean
    return mean([apk(a, p, k) for a, p in zip(actual, predicted)])

Metric applied for ```K=7```:

In [5]:
metric_results = mapk(actual_sorted, preds_sorted, 7)
metric_results

0.866297175189556